# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [43]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys as Teclas
# https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/?tabs=python # ISSO AKI SALVO
# https://selenium-python.readthedocs.io/

# .send_keys() ==> PRA ESCREVER
# .click() ==> PRA CLIKAR
# .get() ==> PRA PEGAR INFORMAÇÃO DO ELEMENTO E TB ABRIR SITE

#----------------------------DOLAR
# Abrir o navegador
navegator = webdriver.Edge()
# Abrir no Google
navegator.get("https://www.google.com/")
# Selecionar Pesquisa e Passar Parametro
navegator.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dolar")
# Mandar Pesquisar
navegator.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]").send_keys(Teclas.ENTER) # Os Comandos tem que ser MAIUSCULOS
# Pegar Cotação
cotação_dolar = navegator.find_element("xpath", '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
# Fechar
navegator.close()

#----------------------------EURO
# Abrir o navegador
navegator = webdriver.Edge()
# Abrir no Google
navegator.get("https://www.google.com/")
# Selecionar Pesquisa e Passar Parametro
navegator.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro")
# Mandar Pesquisar
navegator.find_element("xpath", "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[3]/center/input[1]").send_keys(Teclas.ENTER) # Os Comandos tem que ser MAIUSCULOS
# Pegar Cotação - NUMERO
cotação_euro = navegator.find_element("xpath", '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
# Fechar
navegator.quit()

#----------------------------OURO
# Abrir o navegador
navegator = webdriver.Edge()
# Abrir no Google
navegator.get("https://www.melhorcambio.com/ouro-hoje")
# Pegar Cotação
cotação_ouro = navegator.find_element("xpath", '//*[@id="comercial"]').get_attribute("value")
# TA COM VIRGULA E TEM QUE MUDAR PRA PONTO
cotação_ouro = cotação_ouro.replace(",", ".")
# Fechar
navegator.close() # OU .quit()

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [47]:
import pandas as pd
tabela = pd.read_excel(r"C:\Users\arthu\Downloads\Produtos.xlsx")
tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [45]:
# Atualizar Cotações

D = tabela["Moeda"] == "Dolar"
E = tabela["Moeda"] == "Euro"
O = tabela["Moeda"] == "Ouro"

tabela.loc[D, "Cotação"] = float(cotação_dolar)
tabela.loc[E, "Cotação"] = float(cotação_euro)
tabela.loc[O, "Cotação"] = float(cotação_ouro)

# Preço de Compra = Preço Original X Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# Preço de Venda = Preço de Compra X Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

tabela

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.000000,4999.950000,1.40,6999.930000
1,Carro Renault,4500.00,Euro,5.136189,23112.851247,2.00,46225.702494
2,Notebook Dell,899.99,Dólar,5.000000,4499.950000,1.70,7649.915000
3,IPhone,799.00,Dólar,5.000000,3995.000000,1.70,6791.500000
4,Carro Fiat,3000.00,Euro,5.136189,15408.567498,1.90,29276.278246
5,Celular Xiaomi,480.48,Dólar,5.000000,2402.400000,2.00,4804.800000
6,Joia 20g,20.00,Ouro,285.900000,5718.000000,1.15,6575.700000


### Agora vamos exportar a nova base de preços atualizada

In [46]:
## Colocar index=False pra ele não aparecer na exportação
tabela.to_excel("Produtos Novo.xlsx", index=False)